In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/mse-2-ai-201-b-aiml-a/train.csv")
df

In [ ]:
df1 = pd.read_csv("/kaggle/input/mse-2-ai-201-b-aiml-a/test.csv")
df1.head()

In [ ]:
df.info()

In [ ]:
df1.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
y = df["Class"]


In [ ]:
# le_target = LabelEncoder()
# y = le_target.fit_transform(y.astype(str))
# y
y = y.map({"A":0,"B":1})
y


In [ ]:
y = y.fillna(y.median())
y

In [ ]:
X = df.drop(["Class"],axis = 1)
X.head()

In [ ]:
X = X.fillna(X.mean(numeric_only = True))
X.info()

In [ ]:
def cap_outliers(df,cols,l=1,u = 99):
    for col in cols:
        lv = df[col].quantile(l/100)
        uv = df[col].quantile(u/100)
        df[col] = df[col].clip(lv,uv)
    return df
num_cols = X.select_dtypes(include = ["float64"]).columns
X = cap_outliers(X,num_cols)


In [ ]:
cat_col = "Class"
for col in num_cols:
    plt.figure(figsize = (6,4))
    sns.boxplot(x = cat_col,y = col,data = df)

In [ ]:
sns.heatmap(X.corr())

In [ ]:
sns.pairplot(X)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42,stratify = y)
X_train.head()

In [ ]:
model = RandomForestClassifier(n_estimators = 1000,max_depth = 50,min_samples_split = 2,min_samples_leaf = 1,max_features = 'sqrt')
model.fit(X_train,y_train)


In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(
    objective = "multiclass",
    n_estimators = 1000,
    learning_rate = 0.01,
    num_leaves = 31,
    num_classes = 2,
    colsample_bytree = 0.8,
    reg_alpha = 0.1,
    reg_lambda = 1.0,
    n_jobs = -1,
    random_state = 42
)
clf.fit(X_train,y_train)


In [ ]:
y_p = clf.predict(X_test)
a = accuracy_score(y_test,y_p)
a

In [ ]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test,y_pred)
acc

In [ ]:
test_ids = df1["id"]
x_test = df1.drop(["id"],axis = 1)


In [ ]:
y_pred_f = clf.predict(x_test)
# y_pred_f
y_pred_df = pd.DataFrame(y_pred_f)
y_pred_df

In [ ]:
y_pred1 = y_pred_df[0].map({0.0:"A",1.0:"B"})
y_pred1

In [ ]:
submission = pd.DataFrame({
    "id" : test_ids,
    "Class" : y_pred1
})
submission.to_csv("amit.csv",index = False)
submission.head()